In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/ITM_Project/'

Mounted at /content/drive


### Setting

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00


In [ ]:
!pip install torcheval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 13.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
import torcheval.metrics
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from torcheval.metrics.functional import r2_score

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

### Load XED dataset

In [ ]:
xed = pd.read_csv(path + 'data/XED/XED.tsv', delimiter='\t')

In [ ]:
xed.shape

(17528, 2)

In [ ]:
xed.head()

,sentence,label
0,", ...",1
1,!,"1, 4, 7"
2,... And I don't think we need to discuss the T...,"8, 1"
3,* So get up out of your bed,1
4,A confession that you hired [PERSON] ... and a...,"1, 6"


In [ ]:
list(map(int, str(xed['label'][1]).split(',')))

[1, 4, 7]

In [ ]:
VAscores = [[0.5, 0.5], [0.167, 0.865], [0.698, 0.539], [0.052, 0.775], [0.073, 0.840], [0.980, 0.824], [0.052, 0.288], [0.875, 0.875], [0.888, 0.547]]
print(VAscores[5])

[0.98, 0.824]


### Hyperparameter Setting

In [ ]:
max_len = 256
minibatch_size = 8
learn_rate = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = dataframe.sentence
        self.labels = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        targets = list(map(int, str(self.labels[index]).split(',')))
        scores = [0, 0]
        for i in targets:
            scores[0] += VAscores[i][0]
            scores[1] += VAscores[i][1]
        scores[0] /= len(targets)
        scores[1] /= len(targets)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(scores, dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data=xed.sample(frac=train_size,random_state=1)
test_data=xed.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(xed.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, max_len)
testing_set = SentimentData(test_data, tokenizer, max_len)

FULL Dataset: (17528, 2)
TRAIN Dataset: (14022, 2)
TEST Dataset: (3506, 2)


In [ ]:
train_loader = DataLoader(training_set, batch_size=minibatch_size,
                shuffle=True,)
test_loader = DataLoader(testing_set, batch_size=minibatch_size,
                shuffle=True,)

## Network including Roberta

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.classifier0 = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier1 = torch.nn.Linear(768, 1)
        self.classifier2 = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_roberta = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_roberta[0]
        x = hidden_state[:, 0]
        x = self.classifier0(x)
        x = torch.nn.ReLU()(x)
        x = self.dropout(x)
        output1 = self.classifier1(x)
        output2 = self.classifier2(x)
        return output1, output2

In [ ]:
model = RobertaClass()
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

### Loss Function and Optimizer

In [ ]:
# Creating the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=learn_rate)

### Train & Test

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    nb_tr_steps = 0
    model.train()
    for _,data in tqdm(enumerate(train_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs1, outputs2 = model(ids, mask, token_type_ids)
        outputs1 = outputs1.squeeze()
        outputs2 = outputs2.squeeze()
        loss1 = criterion(outputs1, targets[:,0])
        loss2 = criterion(outputs2, targets[:,1])
        loss_total = loss1 + loss2
        tr_loss += loss_total.item()

        nb_tr_steps += 1

        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss: {loss_step}")

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Training Loss Epoch {epoch}: {epoch_loss}")

    return

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:03,  3.07s/it]

Training Loss: 1.204254150390625


101it [00:37,  2.87it/s]

Training Loss: 0.2938752881076076


201it [01:13,  2.71it/s]

...

Training Loss: 0.029966094906760065


1701it [10:37,  2.67it/s]

Training Loss: 0.03029516533053842


1753it [10:56,  2.67it/s]

Training Loss Epoch: 0.030474956984792693


In [ ]:
def test(model, test_loader):
    model.eval()
    tr_loss=0; nb_tr_steps=0
    total_outputs1 = []
    total_outputs2 = []
    total_targets1 = []
    total_targets2 = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(test_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs1, outputs2 = model(ids, mask, token_type_ids)
            outputs1 = outputs1.squeeze()
            outputs2 = outputs2.squeeze()
            total_outputs1.append(outputs1)
            total_outputs2.append(outputs2)
            total_targets1.append(targets[:,0])
            total_targets2.append(targets[:,1])

            loss1 = criterion(outputs1, targets[:,0])
            loss2 = criterion(outputs2, targets[:,1])
            loss_total = loss1 + loss2
            tr_loss += loss_total.item()

            nb_tr_steps += 1

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Validation Loss: {loss_step}")
    total_outputs1=torch.cat(total_outputs1, dim=0)
    total_outputs2=torch.cat(total_outputs2, dim=0)
    total_targets1=torch.cat(total_targets1, dim=0)
    total_targets2=torch.cat(total_targets2, dim=0)
    score1=r2_score(total_outputs1, total_targets1)
    score2=r2_score(total_outputs2, total_targets2)
    print(f"Valence r2_score: {score1}")
    print(f"Arousal r2_score: {score2}")

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Validation Loss Epoch: {epoch_loss}")

    return epoch_loss

In [ ]:
loss = test(model, test_loader)
print("Loss on test data = %.6f" % loss)

2it [00:00,  7.37it/s]

Validation Loss: 0.23513369262218475


102it [00:12,  8.17it/s]

Validation Loss: 0.11085086265154699


202it [00:24,  7.95it/s]

...

Validation Loss: 0.11248689596587637


402it [00:48,  8.31it/s]

Validation Loss: 0.11035287733238534


439it [00:53,  8.23it/s]

Valence r2_score: 0.3832032084465027
Arousal r2_score: 0.19144612550735474
Validation Loss Epoch: 0.10979011337058556
Loss on test data = 0.109790


### Code Save

In [ ]:
from datetime import date, datetime

In [ ]:
day = date.today().day
current_time = datetime.now().strftime("%H_%M_%S")

output_model_file = 'emotion_recognizer/models/model_epoch_10_' + '%d' % day + '_' + current_time + '.pt'
#output_vocab_file = 'emotion_recognizer/'

model_to_save = model
torch.save(model_to_save, path + output_model_file)
#tokenizer.save_vocabulary(path + output_vocab_file)

print('Model saved')

Model saved


### Sample Test

In [ ]:
sample = pd.read_csv(path + 'data/IMSDb/LaLaLand.tsv', delimiter='\t')
sample.shape

(86, 2)

In [ ]:
sample.head()

,sentence,label
0,Feels so good to be home.,0
1,Stay.,0
2,How's the play going?,0
3,I'm nervous.,0
4,Why?,0


In [ ]:
sample_set = SentimentData(sample, tokenizer, max_len)

In [ ]:
sample_loader = DataLoader(sample_set, batch_size=1,
                shuffle=False,)

In [ ]:
sample_list_v = []
sample_list_a = []
def sample_test(model, sample_loader):
    model.eval()
    index = 0
    with torch.no_grad():
        for _, data in tqdm(enumerate(sample_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            index += 1
            outputs1, outputs2 = model(ids, mask, token_type_ids)
            outputs1 = outputs1.squeeze()
            outputs2 = outputs2.squeeze()
            print('\n')
            print(str(index) + ' v score: ' + str(outputs1.item()))
            print(str(index) + ' a score: ' + str(outputs2.item()))
            sample_list_v.append(outputs1.item()*2 - 1)
            sample_list_a.append(outputs2.item()*2 - 1)

    return

- Or you can load the model

In [ ]:
load_model = torch.load(path + 'emotion_recognizer/models/controlled_model_epoch_10_8_15_15_35.pt')
load_model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
sample_list_v = []
sample_list_a = []
sample_test(load_model, sample_loader)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
4it [00:00, 33.70it/s]



1 v score: 0.9963216185569763
1 a score: 0.8974641561508179


2 v score: 0.3536921739578247
2 a score: 0.5942056179046631


3 v score: 0.8118022680282593
3 a score: 0.6569246649742126


4 v score: 0.10216329991817474
4 a score: 0.8277353644371033


5 v score: 0.573930561542511
5 a score: 0.5557988882064819


6 v score: 0.3201965093612671
6 a score: 0.4290737509727478


7 v score: 0.7222663760185242
7 a score: 0.5518450736999512


13it [00:00, 39.19it/s]



8 v score: 0.14517982304096222
8 a score: 0.8284372091293335


9 v score: 0.233761727809906
9 a score: 0.8477955460548401


10 v score: 0.08076123148202896
10 a score: 0.7726482152938843


11 v score: 0.0909116342663765
11 a score: 0.8454543352127075


12 v score: 0.9577360153198242
12 a score: 0.8708105087280273


13 v score: 0.6707900166511536
13 a score: 0.5278455018997192


14 v score: 0.6498345136642456
14 a score: 0.5470234751701355


15 v score: 0.7815511226654053
15 a score: 0.5227004289627075


16 v score: 0.623514711856842
16 a score: 0.5326461791992188


17 v score: 0.46321341395378113
17 a score: 0.486549437046051


23it [00:00, 43.36it/s]

...



70 v score: 0.7682164311408997
70 a score: 0.5964818000793457


71 v score: 0.15894080698490143
71 a score: 0.7888581156730652


72 v score: 0.5819903016090393
72 a score: 0.5420746207237244


73 v score: 0.6878426671028137
73 a score: 0.5469855666160583


74 v score: 0.05614693462848663
74 a score: 0.39353886246681213


75 v score: 0.8962417244911194
75 a score: 0.6882773041725159


76 v score: 0.15713699162006378
76 a score: 0.8204066157341003


77 v score: 0.5087448954582214
77 a score: 0.6110371947288513


83it [00:02, 38.53it/s]



78 v score: 0.05827165022492409
78 a score: 0.4931240379810333


79 v score: 0.985072135925293
79 a score: 0.9004055261611938


80 v score: 0.8989283442497253
80 a score: 0.8028223514556885


81 v score: 0.29236093163490295
81 a score: 0.8249002695083618


82 v score: 0.11911644041538239
82 a score: 0.8257876634597778


83 v score: 0.9182103276252747
83 a score: 0.7313860654830933


84 v score: 0.6667730212211609
84 a score: 0.8079032897949219


85 v score: 0.5035713911056519
85 a score: 0.48398423194885254



86it [00:02, 39.52it/s]


86 v score: 0.46076852083206177
86 a score: 0.6095282435417175


In [ ]:
print(sample_list_v)
print(sample_list_a)

[0.9926432371139526, -0.2926156520843506, 0.6236045360565186, -0.7956734001636505, 0.14786112308502197, -0.3596069812774658, 0.44453275203704834, -0.7096403539180756, -0.532476544380188, -0.8384775370359421, -0.818176731467247, 0.9154720306396484, 0.34158003330230713, 0.2996690273284912, 0.5631022453308105, 0.24702942371368408, -0.07357317209243774, 0.2682877779006958, 0.03294110298156738, -0.4456125497817993, -0.8747609555721283, -0.8308763951063156, 0.14897477626800537, 0.44336986541748047, 0.3545706272125244, -0.9142786487936974, -0.310519814491272, -0.900969922542572, 0.5477551221847534, 0.4033123254776001, 0.39501047134399414, 0.3378419876098633, 0.32086026668548584, 0.23952019214630127, 0.45173776149749756, 0.3589440584182739, -0.1449933648109436, 0.5512440204620361, 0.8561397790908813, -0.5920479893684387, 0.4114440679550171, 0.4893397092819214, 0.47729194164276123, -0.7950875014066696, -0.49690961837768555, 0.47323596477508545, 0.08108675479888916, 0.8245207071304321, -0.684594

# Trying Various Datasets

## Additional Neutral Dataset (4000)

In [ ]:
xed_neu = pd.read_csv(path + 'data/XED/neu_ner_4000.tsv', delimiter='\t')

In [ ]:
xed_neu.shape

(4000, 2)

In [ ]:
xed_neu.tail()

,sentence,label
3995,You 're excused .,0
3996,A perfect murder .,0
3997,Plus the [ORGANIZATION] .,0
3998,I do better with women and they do n't eat cig...,0
3999,What are you doing ?,0


In [ ]:
train_size = 0.8
xed_neu_added = pd.concat([xed, xed_neu]).reset_index(drop=True)
neu_train_data=xed_neu_added.sample(frac=train_size,random_state=1)
neu_test_data=xed_neu_added.drop(neu_train_data.index).reset_index(drop=True)
neu_train_data = neu_train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(xed_neu_added.shape))
print("TRAIN Dataset: {}".format(neu_train_data.shape))
print("TEST Dataset: {}".format(neu_test_data.shape))

neu_training_set = SentimentData(neu_train_data, tokenizer, max_len)
neu_testing_set = SentimentData(neu_test_data, tokenizer, max_len)

FULL Dataset: (21528, 2)
TRAIN Dataset: (17222, 2)
TEST Dataset: (3027, 2)


In [ ]:
neu_train_loader = DataLoader(neu_training_set, batch_size=minibatch_size,
                shuffle=True,)
neu_test_loader = DataLoader(neu_testing_set, batch_size=minibatch_size,
                shuffle=True,)

In [ ]:
neu_model = RobertaClass()
neu_model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = neu_model.parameters(), lr=learn_rate)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    nb_tr_steps = 0
    neu_model.train()
    for _,data in tqdm(enumerate(neu_train_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs1, outputs2 = neu_model(ids, mask, token_type_ids)
        outputs1 = outputs1.squeeze()
        outputs2 = outputs2.squeeze()
        loss1 = criterion(outputs1, targets[:,0])
        loss2 = criterion(outputs2, targets[:,1])
        loss_total = loss1 + loss2
        tr_loss += loss_total.item()

        nb_tr_steps += 1

        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss: {loss_step}")

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Training Loss Epoch: {epoch_loss}")

    return

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

1it [00:00,  5.10it/s]

Training Loss: 0.027969498187303543
Training R2 Score per 5000 steps: 5.839202925562859


101it [00:34,  2.86it/s]

Training Loss: 0.025607365663025048
Training R2 Score per 5000 steps: 8.567555827817115


201it [01:09,  2.76it/s]

...

1701it [10:04,  2.82it/s]

Training Loss: 0.015387846658888904
Training R2 Score per 5000 steps: 9.936813495479424


1801it [10:40,  2.81it/s]

Training Loss: 0.015389388419640174
Training R2 Score per 5000 steps: 9.941642721499555


1859it [11:00,  2.79it/s]

In [ ]:
def test(neu_model, neu_test_loader):
    neu_model.eval()
    tr_loss=0; nb_tr_steps=0;
    total_outputs1 = []
    total_outputs2 = []
    total_targets1 = []
    total_targets2 = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(neu_test_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs1, outputs2 = neu_model(ids, mask, token_type_ids)
            outputs1 = outputs1.squeeze()
            outputs2 = outputs2.squeeze()
            total_outputs1.append(outputs1)
            total_outputs2.append(outputs2)
            total_targets1.append(targets[:,0])
            total_targets2.append(targets[:,1])

            loss1 = criterion(outputs1, targets[:,0])
            loss2 = criterion(outputs2, targets[:,1])
            loss_total = loss1 + loss2
            tr_loss += loss_total.item()

            nb_tr_steps += 1

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Validation Loss: {loss_step}")

    total_outputs1=torch.cat(total_outputs1, dim=0)
    total_outputs2=torch.cat(total_outputs2, dim=0)
    total_targets1=torch.cat(total_targets1, dim=0)
    total_targets2=torch.cat(total_targets2, dim=0)
    score1=r2_score(total_outputs1, total_targets1)
    score2=r2_score(total_outputs2, total_targets2)
    print(f"Valence r2_score: {score1}")
    print(f"Arousal r2_score: {score2}")

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Validation Loss Epoch: {epoch_loss}")

    return epoch_loss

In [ ]:
loss = test(neu_model, neu_test_loader)
print("Loss on neutral-added test data = %.6f" % loss)

2it [00:00,  6.75it/s]

Validation Loss: 0.1911604255437851
Validation R2 Score per 100 steps: -1.2817300856113434


102it [00:11,  9.11it/s]

Validation Loss: 0.10853660317820192
Validation R2 Score per 100 steps: -2.6921388093787844


202it [00:22,  8.86it/s]

Validation Loss: 0.10494813329971103
Validation R2 Score per 100 steps: -1.220435804840344


302it [00:33,  8.85it/s]

Validation Loss: 0.10214696224450917
Validation R2 Score per 100 steps: -0.7252779043592091


379it [00:42,  8.93it/s]


r2_score 1: 0.4160304069519043
r2_score 2: 0.1538705825805664
Validation Loss Epoch: 0.10402276580121202
Validation R2 Score Epoch: -1.0267090917539392
Loss on neutral-added test data = 0.104023


In [ ]:
sample_test(neu_model, sample_loader)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
8it [00:00, 36.77it/s]



1 v score: 0.9571362733840942
1 a score: 0.7049903273582458


2 v score: 0.27462685108184814
2 a score: 0.6979231238365173


3 v score: 0.6763144731521606
3 a score: 0.6347838640213013


4 v score: 0.16947701573371887
4 a score: 0.6700631976127625


5 v score: 0.5690644383430481
5 a score: 0.612883448600769


6 v score: 0.3602908253669739
6 a score: 0.5951213836669922


7 v score: 0.4764261841773987
7 a score: 0.689078688621521


8 v score: 0.23529648780822754
8 a score: 0.7313960790634155




20it [00:00, 51.20it/s]

9 v score: 0.2513871192932129
9 a score: 0.7729743123054504


10 v score: 0.1852000504732132
10 a score: 0.5718061327934265


11 v score: 0.08633388578891754
11 a score: 0.7615430355072021


12 v score: 0.7751166224479675
12 a score: 0.7114263772964478


13 v score: 0.9012032747268677
13 a score: 0.7126465439796448


14 v score: 0.6817417740821838
14 a score: 0.6649984121322632


15 v score: 0.6469689607620239
15 a score: 0.5782835483551025


16 v score: 0.5437071919441223
16 a score: 0.6065823435783386


17 v score: 0.4853411316871643
17 a score: 0.4770582616329193


18 v score: 0.5619469285011292
18 a score: 0.6214231848716736


19 v score: 0.5039831399917603
19 a score: 0.5201848745346069


20 v score: 0.6175476312637329
20 a score: 0.6385827660560608


21 v score: 0.28246963024139404
21 a score: 0.3798726499080658


32it [00:00, 56.01it/s]

...



72 v score: 0.6324983239173889
72 a score: 0.6485412120819092


73 v score: 0.619265615940094
73 a score: 0.6195147633552551


74 v score: 0.2965611517429352
74 a score: 0.7104666233062744


75 v score: 0.5490533113479614
75 a score: 0.6414245367050171


76 v score: 0.6834088563919067
76 a score: 0.737516462802887


77 v score: 0.3014678657054901
77 a score: 0.699726939201355


78 v score: 0.09824060648679733
78 a score: 0.6351084113121033


79 v score: 0.949346125125885
79 a score: 0.821892261505127


80 v score: 0.25609642267227173
80 a score: 0.7774156928062439


81 v score: 0.4032163918018341
81 a score: 0.8066997528076172


82 v score: 0.24715767800807953
82 a score: 0.841484785079956


83 v score: 0.496239572763443
83 a score: 0.6044822931289673


84 v score: 0.3463115990161896
84 a score: 0.7620711922645569


94it [00:01, 59.12it/s]



85 v score: 0.3842950463294983
85 a score: 0.5783076286315918


86 v score: 0.43462201952934265
86 a score: 0.6140514016151428


87 v score: 0.035148054361343384
87 a score: 0.2367057204246521


88 v score: 0.10898688435554504
88 a score: 0.6600242257118225


89 v score: 0.09664370119571686
89 a score: 0.5390317440032959


90 v score: 0.09627901017665863
90 a score: 0.5747007131576538


91 v score: 0.07848870009183884
91 a score: 0.42829954624176025


92 v score: 0.06509511917829514
92 a score: 0.43056339025497437


93 v score: 0.10923251509666443
93 a score: 0.4585457146167755


94 v score: 0.6113942861557007
94 a score: 0.6221528053283691


95 v score: 0.09669091552495956
95 a score: 0.5699253678321838


96 v score: 0.1956179141998291
96 a score: 0.7365403771400452


97 v score: 0.09190623462200165
97 a score: 0.48251768946647644


99it [00:01, 55.99it/s]



98 v score: 0.3600294291973114
98 a score: 0.5286602973937988


99 v score: 0.21921898424625397
99 a score: 0.5504127740859985


In [ ]:
print(sample_list_v)
print(sample_list_a)

[0.9142725467681885, -0.4507462978363037, 0.3526289463043213, -0.6610459685325623, 0.1381288766860962, -0.27941834926605225, -0.04714763164520264, -0.5294070243835449, -0.4972257614135742, -0.6295998990535736, -0.8273322284221649, 0.5502332448959351, 0.8024065494537354, 0.3634835481643677, 0.29393792152404785, 0.08741438388824463, -0.029317736625671387, 0.1238938570022583, 0.007966279983520508, 0.23509526252746582, -0.4350607395172119, -0.1766601800918579, -0.1786927580833435, -0.14548563957214355, -0.01918083429336548, -0.5174876749515533, 0.06729495525360107, -0.6698314845561981, -0.21532851457595825, -0.09094816446304321, 0.0381702184677124, 0.10097050666809082, 0.361011266708374, 0.06769943237304688, -0.016296029090881348, 0.1391695737838745, -0.44275200366973877, 0.023219943046569824, 0.3691897392272949, -0.34884363412857056, -0.0038782358169555664, 0.26021885871887207, 0.4229319095611572, -0.7601215094327927, -0.4580203890800476, 0.47004735469818115, 0.3856525421142578, 0.1973649

In [ ]:
from datetime import date, datetime

In [ ]:
day = date.today().day
current_time = datetime.now().strftime("%H_%M_%S")

output_model_file = 'emotion_recognizer/models/neu_model_4000_' + '%d' % day + '_' + current_time + '.pt'
#output_vocab_file = 'emotion_recognizer/'

model_to_save = neu_model
torch.save(model_to_save, path + output_model_file)
#tokenizer.save_vocabulary(path + output_vocab_file)

print('NEU_Model saved')

NEU_Model saved


## Additional Neutral Dataset (total neutral set)

In [ ]:
xed_neu_total = pd.read_csv(path + 'data/XED/neu_ner.tsv', delimiter='\t')

In [ ]:
xed_neu_total.shape

(9539, 2)

In [ ]:
xed_neu_total.tail()

,sentence,label
9534,A baseball bat would be nice .,0
9535,Give him what he wants .,0
9536,Or perhaps you 'd know what you did with the m...,0
9537,Why are n't they telling us anything ?,0
9538,That was a spontaneous refreshing reaction . I...,0


In [ ]:
train_size = 0.8
xed_neu_total_added = pd.concat([xed, xed_neu_total]).reset_index(drop=True)
neu_total_train_data=xed_neu_total_added.sample(frac=train_size,random_state=1)
neu_total_test_data=xed_neu_total_added.drop(neu_total_train_data.index).reset_index(drop=True)
neu_total_train_data = neu_total_train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(xed_neu_total_added.shape))
print("TRAIN Dataset: {}".format(neu_total_train_data.shape))
print("TEST Dataset: {}".format(neu_total_test_data.shape))

neu_total_training_set = SentimentData(neu_total_train_data, tokenizer, max_len)
neu_total_testing_set = SentimentData(neu_total_test_data, tokenizer, max_len)

FULL Dataset: (27067, 2)
TRAIN Dataset: (21654, 2)
TEST Dataset: (5413, 2)


In [ ]:
neu_total_train_loader = DataLoader(neu_total_training_set, batch_size=minibatch_size,
                shuffle=True,)
neu_total_test_loader = DataLoader(neu_total_testing_set, batch_size=minibatch_size,
                shuffle=True,)

In [ ]:
neu_total_model = RobertaClass()
neu_total_model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = neu_total_model.parameters(), lr=learn_rate)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    nb_tr_steps = 0
    neu_total_model.train()
    for _,data in tqdm(enumerate(neu_total_train_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs1, outputs2 = neu_total_model(ids, mask, token_type_ids)
        outputs1 = outputs1.squeeze()
        outputs2 = outputs2.squeeze()
        loss1 = criterion(outputs1, targets[:,0])
        loss2 = criterion(outputs2, targets[:,1])
        loss_total = loss1 + loss2
        tr_loss += loss_total.item()

        nb_tr_steps += 1

        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss: {loss_step}")

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Training Loss Epoch: {epoch_loss}")

    return

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

1it [00:00,  4.20it/s]

Training Loss: 0.5888514518737793


101it [00:33,  2.96it/s]

Training Loss: 0.21954280956841932


201it [01:07,  2.89it/s]

...

Training Loss: 0.02429153907248125


2701it [16:18,  2.75it/s]

Training Loss: 0.024325484445141692


2707it [16:20,  2.76it/s]

Training Loss Epoch: 0.024361282299991075


In [ ]:
def test(neu_total_model, neu_total_test_loader):
    neu_total_model.eval()
    tr_loss=0; nb_tr_steps=0;
    total_outputs1 = []
    total_outputs2 = []
    total_targets1 = []
    total_targets2 = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(neu_total_test_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs1, outputs2 = neu_total_model(ids, mask, token_type_ids)
            outputs1 = outputs1.squeeze()
            outputs2 = outputs2.squeeze()
            total_outputs1.append(outputs1)
            total_outputs2.append(outputs2)
            total_targets1.append(targets[:,0])
            total_targets2.append(targets[:,1])

            loss1 = criterion(outputs1, targets[:,0])
            loss2 = criterion(outputs2, targets[:,1])
            loss_total = loss1 + loss2
            tr_loss += loss_total.item()

            nb_tr_steps += 1

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Validation Loss: {loss_step}")

    total_outputs1=torch.cat(total_outputs1, dim=0)
    total_outputs2=torch.cat(total_outputs2, dim=0)
    total_targets1=torch.cat(total_targets1, dim=0)
    total_targets2=torch.cat(total_targets2, dim=0)
    score1=r2_score(total_outputs1, total_targets1)
    score2=r2_score(total_outputs2, total_targets2)
    print(f"Valence r2_score: {score1}")
    print(f"Arousal r2_score: {score2}")

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Validation Loss Epoch: {epoch_loss}")

    return epoch_loss

In [ ]:
loss = test(neu_total_model, neu_total_test_loader)
print("Loss on neutral-added test data = %.6f" % loss)

2it [00:00,  6.04it/s]

Validation Loss: 0.20067059993743896


102it [00:11,  9.22it/s]

Validation Loss: 0.08388989014864558


202it [00:22,  9.06it/s]

...

Validation Loss: 0.08379473146367186


602it [01:06,  9.17it/s]

Validation Loss: 0.08308269465171705


677it [01:14,  9.03it/s]


Valence r2_score: 0.3058432340621948
Arousal r2_score: 0.2682368755340576
Validation Loss Epoch: 0.08378115468493427
Loss on neutral-added test data = 0.083781


In [ ]:
sample_test(neu_total_model, sample_loader)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
8it [00:00, 36.77it/s]



1 v score: 0.9571362733840942
1 a score: 0.7049903273582458


2 v score: 0.27462685108184814
2 a score: 0.6979231238365173


3 v score: 0.6763144731521606
3 a score: 0.6347838640213013


4 v score: 0.16947701573371887
4 a score: 0.6700631976127625


5 v score: 0.5690644383430481
5 a score: 0.612883448600769


6 v score: 0.3602908253669739
6 a score: 0.5951213836669922


7 v score: 0.4764261841773987
7 a score: 0.689078688621521


8 v score: 0.23529648780822754
8 a score: 0.7313960790634155




20it [00:00, 51.20it/s]

9 v score: 0.2513871192932129
9 a score: 0.7729743123054504


10 v score: 0.1852000504732132
10 a score: 0.5718061327934265


11 v score: 0.08633388578891754
11 a score: 0.7615430355072021


12 v score: 0.7751166224479675
12 a score: 0.7114263772964478


13 v score: 0.9012032747268677
13 a score: 0.7126465439796448


14 v score: 0.6817417740821838
14 a score: 0.6649984121322632


15 v score: 0.6469689607620239
15 a score: 0.5782835483551025


16 v score: 0.5437071919441223
16 a score: 0.6065823435783386


17 v score: 0.4853411316871643
17 a score: 0.4770582616329193


18 v score: 0.5619469285011292
18 a score: 0.6214231848716736


19 v score: 0.5039831399917603
19 a score: 0.5201848745346069


20 v score: 0.6175476312637329
20 a score: 0.6385827660560608


21 v score: 0.28246963024139404
21 a score: 0.3798726499080658


32it [00:00, 56.01it/s]

...



72 v score: 0.6324983239173889
72 a score: 0.6485412120819092


73 v score: 0.619265615940094
73 a score: 0.6195147633552551


74 v score: 0.2965611517429352
74 a score: 0.7104666233062744


75 v score: 0.5490533113479614
75 a score: 0.6414245367050171


76 v score: 0.6834088563919067
76 a score: 0.737516462802887


77 v score: 0.3014678657054901
77 a score: 0.699726939201355


78 v score: 0.09824060648679733
78 a score: 0.6351084113121033


79 v score: 0.949346125125885
79 a score: 0.821892261505127


80 v score: 0.25609642267227173
80 a score: 0.7774156928062439


81 v score: 0.4032163918018341
81 a score: 0.8066997528076172


82 v score: 0.24715767800807953
82 a score: 0.841484785079956


83 v score: 0.496239572763443
83 a score: 0.6044822931289673


84 v score: 0.3463115990161896
84 a score: 0.7620711922645569


94it [00:01, 59.12it/s]



85 v score: 0.3842950463294983
85 a score: 0.5783076286315918


86 v score: 0.43462201952934265
86 a score: 0.6140514016151428


87 v score: 0.035148054361343384
87 a score: 0.2367057204246521


88 v score: 0.10898688435554504
88 a score: 0.6600242257118225


89 v score: 0.09664370119571686
89 a score: 0.5390317440032959


90 v score: 0.09627901017665863
90 a score: 0.5747007131576538


91 v score: 0.07848870009183884
91 a score: 0.42829954624176025


92 v score: 0.06509511917829514
92 a score: 0.43056339025497437


93 v score: 0.10923251509666443
93 a score: 0.4585457146167755


94 v score: 0.6113942861557007
94 a score: 0.6221528053283691


95 v score: 0.09669091552495956
95 a score: 0.5699253678321838


96 v score: 0.1956179141998291
96 a score: 0.7365403771400452


97 v score: 0.09190623462200165
97 a score: 0.48251768946647644


99it [00:01, 55.99it/s]



98 v score: 0.3600294291973114
98 a score: 0.5286602973937988


99 v score: 0.21921898424625397
99 a score: 0.5504127740859985


In [ ]:
print(sample_list_v)
print(sample_list_a)

[0.9142725467681885, -0.4507462978363037, 0.3526289463043213, -0.6610459685325623, 0.1381288766860962, -0.27941834926605225, -0.04714763164520264, -0.5294070243835449, -0.4972257614135742, -0.6295998990535736, -0.8273322284221649, 0.5502332448959351, 0.8024065494537354, 0.3634835481643677, 0.29393792152404785, 0.08741438388824463, -0.029317736625671387, 0.1238938570022583, 0.007966279983520508, 0.23509526252746582, -0.4350607395172119, -0.1766601800918579, -0.1786927580833435, -0.14548563957214355, -0.01918083429336548, -0.5174876749515533, 0.06729495525360107, -0.6698314845561981, -0.21532851457595825, -0.09094816446304321, 0.0381702184677124, 0.10097050666809082, 0.361011266708374, 0.06769943237304688, -0.016296029090881348, 0.1391695737838745, -0.44275200366973877, 0.023219943046569824, 0.3691897392272949, -0.34884363412857056, -0.0038782358169555664, 0.26021885871887207, 0.4229319095611572, -0.7601215094327927, -0.4580203890800476, 0.47004735469818115, 0.3856525421142578, 0.1973649

In [ ]:
from datetime import date, datetime

In [ ]:
day = date.today().day
current_time = datetime.now().strftime("%H_%M_%S")

output_model_file = 'emotion_recognizer/models/neu_total_model_' + '%d' % day + '_' + current_time + '.pt'
#output_vocab_file = 'emotion_recognizer/'

model_to_save = neu_total_model
torch.save(model_to_save, path + output_model_file)
#tokenizer.save_vocabulary(path + output_vocab_file)

print('NEU_TOTAL_Model saved')

NEU_TOTAL_Model saved


### Single Label Dataset

In [ ]:
train_size = 0.8
xed_neu_added = pd.concat([xed, xed_neu])
xed_neu_added.head()

,sentence,label
0,", ...",1
1,!,"1, 4, 7"
2,... And I don't think we need to discuss the T...,"8, 1"
3,* So get up out of your bed,1
4,A confession that you hired [PERSON] ... and a...,"1, 6"


In [ ]:
is_single_labeled = xed_neu_added['label'].apply(lambda x: len(str(x))) == 1
xed_single_neu = xed_neu_added[is_single_labeled].reset_index(drop=True)
xed_single_neu.head()

,sentence,label
0,", ...",1
1,* So get up out of your bed,1
2,A dead man has one half - hour to raise his ro...,1
3,A guy that's talking about he's gonna solve al...,1
4,A guy who was a pain in the neck even before I...,1


In [ ]:
train_size = 0.8
single_neu_train_data=xed_single_neu.sample(frac=train_size,random_state=1)
single_neu_test_data=xed_single_neu.drop(single_neu_train_data.index).reset_index(drop=True)
single_neu_train_data = single_neu_train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(xed_single_neu.shape))
print("TRAIN Dataset: {}".format(single_neu_train_data.shape))
print("TEST Dataset: {}".format(single_neu_test_data.shape))

single_neu_training_set = SentimentData(single_neu_train_data, tokenizer, max_len)
single_neu_testing_set = SentimentData(single_neu_test_data, tokenizer, max_len)

FULL Dataset: (17655, 2)
TRAIN Dataset: (14124, 2)
TEST Dataset: (3531, 2)


In [ ]:
single_neu_train_loader = DataLoader(single_neu_training_set, batch_size=minibatch_size,
                shuffle=True,)
single_neu_test_loader = DataLoader(single_neu_testing_set, batch_size=minibatch_size,
                shuffle=True,)

In [ ]:
single_neu_model = RobertaClass()
single_neu_model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = single_neu_model.parameters(), lr=learn_rate)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    nb_tr_steps = 0
    single_neu_model.train()
    for _,data in tqdm(enumerate(single_neu_train_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs1, outputs2 = single_neu_model(ids, mask, token_type_ids)
        outputs1 = outputs1.squeeze()
        outputs2 = outputs2.squeeze()
        loss1 = criterion(outputs1, targets[:,0])
        loss2 = criterion(outputs2, targets[:,1])
        loss_total = loss1 + loss2
        tr_loss += loss_total.item()

        nb_tr_steps += 1

        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss: {loss_step}")

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Training Loss Epoch: {epoch_loss}")

    return

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  4.43it/s]

Training Loss: 1.504380702972412


101it [00:38,  2.58it/s]

Training Loss: 0.2957969507811093


201it [01:15,  2.70it/s]

...

Training Loss: 0.027647996595452803


1701it [10:36,  2.67it/s]

Training Loss: 0.027695868255051485


1766it [11:00,  2.67it/s]

Training Loss Epoch: 0.027733754853260233


In [ ]:
def test(single_neu_model, single_neu_test_loader):
    single_neu_model.eval()
    tr_loss=0; nb_tr_steps=0;
    total_outputs1 = []
    total_outputs2 = []
    total_targets1 = []
    total_targets2 = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(single_neu_test_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs1, outputs2 = single_neu_model(ids, mask, token_type_ids)
            outputs1 = outputs1.squeeze()
            outputs2 = outputs2.squeeze()
            total_outputs1.append(outputs1)
            total_outputs2.append(outputs2)
            total_targets1.append(targets[:,0])
            total_targets2.append(targets[:,1])

            loss1 = criterion(outputs1, targets[:,0])
            loss2 = criterion(outputs2, targets[:,1])
            loss_total = loss1 + loss2
            tr_loss += loss_total.item()

            nb_tr_steps += 1

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Validation Loss: {loss_step}")

    total_outputs1=torch.cat(total_outputs1, dim=0)
    total_outputs2=torch.cat(total_outputs2, dim=0)
    total_targets1=torch.cat(total_targets1, dim=0)
    total_targets2=torch.cat(total_targets2, dim=0)
    score1=r2_score(total_outputs1, total_targets1)
    score2=r2_score(total_outputs2, total_targets2)
    print(f"Valence r2_score: {score1}")
    print(f"Arousal r2_score: {score2}")

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Validation Loss Epoch: {epoch_loss}")

    return epoch_loss

In [ ]:
loss = test(single_neu_model, single_neu_test_loader)
print("Loss on neutral-added test data = %.6f" % loss)

2it [00:00,  4.02it/s]

Validation Loss: 0.10715913772583008
Validation r2_score per 100 steps: 2.4704620242118835


102it [00:12,  8.24it/s]

Validation Loss: 0.101297635488249
Validation r2_score per 100 steps: 2.8802751093217642


202it [00:24,  8.08it/s]

...

Validation Loss: 0.10103422988042161
Validation r2_score per 100 steps: 3.6869068329912484


402it [00:49,  8.24it/s]

Validation Loss: 0.09906386778538377
Validation r2_score per 100 steps: 3.865865430332479


442it [00:54,  8.09it/s]


Valence r2_score: 0.3805994391441345
Arousal r2_score: 0.23063892126083374
Validation Loss Epoch: 0.09951480688320208
Validation r2_score Epoch: 3.881149506710704
Loss on neutral-added test data = 0.099515


In [ ]:
sample_list_v = []
sample_list_a = []
sample_test(single_neu_model, sample_loader)

3it [00:00, 26.96it/s]



1 v score: 1.038368821144104
1 a score: 0.887800931930542


2 v score: 0.6286042332649231
2 a score: 0.5864723324775696


3 v score: 0.8184511661529541
3 a score: 0.7452869415283203


4 v score: 0.38264647126197815
4 a score: 0.7173305153846741


5 v score: 0.6691921949386597
5 a score: 0.6085864901542664


6 v score: 0.5207858681678772
6 a score: 0.5556479692459106


7 v score: 0.49302393198013306

11it [00:00, 30.34it/s]


7 a score: 0.7548651695251465


8 v score: 0.4335598349571228
8 a score: 0.7552454471588135


9 v score: 0.42360085248947144
9 a score: 0.8023850321769714


10 v score: 0.4421999454498291
10 a score: 0.7215777039527893


11 v score: 0.21059264242649078
11 a score: 0.7094810605049133


12 v score: 0.9099777936935425
12 a score: 0.8154342770576477


13 v score: 0.940150797367096
13 a score: 0.7507522106170654


19it [00:00, 28.90it/s]

...



87 v score: 0.17144550383090973
87 a score: 0.27799153327941895


88 v score: 0.3369656801223755
88 a score: 0.6989170908927917


89 v score: 0.2866823673248291
89 a score: 0.5779498219490051


90 v score: 0.3118418753147125
90 a score: 0.5682258009910583


91 v score: 0.3892100751399994
91 a score: 0.657982349395752


92 v score: 0.25811588764190674
92 a score: 0.38972586393356323


96it [00:03, 25.57it/s]



93 v score: 0.31746259331703186
93 a score: 0.4855206608772278


94 v score: 0.7040156126022339
94 a score: 0.6813846230506897


95 v score: 0.22995559871196747
95 a score: 0.5819199681282043


96 v score: 0.4513249397277832
96 a score: 0.7534034848213196


97 v score: 0.30399346351623535
97 a score: 0.5115686655044556


98 v score: 0.3976786136627197
98 a score: 0.6150172352790833


99it [00:03, 29.65it/s]



99 v score: 0.30365705490112305
99 a score: 0.5328202247619629


In [ ]:
print(sample_list_v)
print(sample_list_a)

[1.076737642288208, 0.2572084665298462, 0.6369023323059082, -0.2347070574760437, 0.33838438987731934, 0.041571736335754395, -0.013952136039733887, -0.1328803300857544, -0.15279829502105713, -0.1156001091003418, -0.5788147151470184, 0.819955587387085, 0.8803015947341919, 0.5623655319213867, 0.4349280595779419, 0.14289915561676025, 0.09391582012176514, 0.327206015586853, 0.26203203201293945, 0.49101758003234863, -0.21897757053375244, 0.09392380714416504, -0.05144155025482178, 0.04985165596008301, 0.32961714267730713, -0.1560744047164917, 0.18800604343414307, -0.31820768117904663, -0.061490774154663086, 0.03464806079864502, 0.09996414184570312, 0.18671166896820068, 0.31890344619750977, 0.2131328582763672, 0.04874897003173828, 0.1983867883682251, -0.087460458278656, 0.09650301933288574, 0.7021772861480713, -0.2885406017303467, 0.12269008159637451, 0.33519506454467773, 0.649200439453125, -0.2865409851074219, -0.2052130103111267, 0.7384737730026245, 0.3369389772415161, 0.3156137466430664, 0.

In [ ]:
from datetime import date, datetime

In [ ]:
day = date.today().day
current_time = datetime.now().strftime("%H_%M_%S")

output_model_file = 'emotion_recognizer/models/single_neu_model_4000_epoch_10_' + '%d' % day + '_' + current_time + '.pt'
#output_vocab_file = 'emotion_recognizer/'

model_to_save = single_neu_model
torch.save(model_to_save, path + output_model_file)
#tokenizer.save_vocabulary(path + output_vocab_file)

print('NEU_Model saved')

NEU_Model saved


### Distribution Controlled Dataset

In [ ]:
train_size = 0.8
xed_neu_added = pd.concat([xed, xed_neu])
xed_neu_added.head()

,sentence,label
0,", ...",1
1,!,"1, 4, 7"
2,... And I don't think we need to discuss the T...,"8, 1"
3,* So get up out of your bed,1
4,A confession that you hired [PERSON] ... and a...,"1, 6"


In [ ]:
is_single_labeled = xed['label'].apply(lambda x: len(str(x))) == 1
xed_single = xed[is_single_labeled].reset_index(drop=True)
xed_single.tail()

,sentence,label
13650,Your opinion might be valuable .,8
13651,Your orders .,8
13652,Your ship's been in lots of battles .,8
13653,"Your wine , your Majesty .",8
13654,Yours ?,8


In [ ]:
is_anger = xed['label'].apply(lambda x: str(x)) == '1'
xed_anger = xed[is_anger].reset_index(drop=True)
xed_anger_data = xed_anger.sample(frac=0.5, random_state=20).reset_index(drop=True)

is_antic = xed['label'].apply(lambda x: str(x)) == '2'
xed_antic = xed[is_antic].reset_index(drop=True)

is_disgust = xed['label'].apply(lambda x: str(x)) == '3'
xed_disgust = xed[is_disgust].reset_index(drop=True)

is_fear = xed['label'].apply(lambda x: str(x)) == '4'
xed_fear = xed[is_fear].reset_index(drop=True)
xed_fear_data = xed_fear.sample(frac=0.5, random_state=20).reset_index(drop=True)

is_joy = xed['label'].apply(lambda x: str(x)) == '5'
xed_joy = xed[is_joy].reset_index(drop=True)
xed_joy_data = xed_joy.sample(frac=0.5, random_state=20).reset_index(drop=True)

is_sad = xed['label'].apply(lambda x: str(x)) == '6'
xed_sad = xed[is_sad].reset_index(drop=True)

is_surp = xed['label'].apply(lambda x: str(x)) == '7'
xed_surp = xed[is_surp].reset_index(drop=True)
xed_surp_data = xed_surp.sample(frac=0.5, random_state=20).reset_index(drop=True)

is_trust = xed['label'].apply(lambda x: str(x)) == '8'
xed_trust = xed[is_trust].reset_index(drop=True)

xed_controlled = pd.concat([xed_anger_data, xed_antic, xed_disgust, xed_fear_data, xed_joy_data, xed_sad, xed_surp_data, xed_trust, xed_neu]).reset_index(drop=True)
xed_controlled.tail()

,sentence,label
14114,You 're excused .,0
14115,A perfect murder .,0
14116,Plus the [ORGANIZATION] .,0
14117,I do better with women and they do n't eat cig...,0
14118,What are you doing ?,0


In [ ]:
train_size = 0.8

controlled_train_data=xed_controlled.sample(frac=train_size,random_state=1)
controlled_test_data=xed_controlled.drop(controlled_train_data.index).reset_index(drop=True)
controlled_train_data = controlled_train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(xed_controlled.shape))
print("TRAIN Dataset: {}".format(controlled_train_data.shape))
print("TEST Dataset: {}".format(controlled_test_data.shape))

controlled_training_set = SentimentData(controlled_train_data, tokenizer, max_len)
controlled_testing_set = SentimentData(controlled_test_data, tokenizer, max_len)

FULL Dataset: (14119, 2)
TRAIN Dataset: (11295, 2)
TEST Dataset: (2824, 2)


In [ ]:
controlled_train_loader = DataLoader(controlled_training_set, batch_size=minibatch_size,
                shuffle=True,)
controlled_test_loader = DataLoader(controlled_testing_set, batch_size=minibatch_size,
                shuffle=True,)

In [ ]:
controlled_model = RobertaClass()
controlled_model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

##### - Or you may load the model

In [ ]:
controlled_model = torch.load(path + 'emotion_recognizer/models/controlled_model_epoch_10_8_15_15_35.pt')
controlled_model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = controlled_model.parameters(), lr=learn_rate)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    nb_tr_steps = 0
    controlled_model.train()
    for _,data in tqdm(enumerate(controlled_train_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs1, outputs2 = controlled_model(ids, mask, token_type_ids)
        outputs1 = outputs1.squeeze()
        outputs2 = outputs2.squeeze()
        loss1 = criterion(outputs1, targets[:,0])
        loss2 = criterion(outputs2, targets[:,1])
        loss_total = loss1 + loss2
        tr_loss += loss_total.item()

        nb_tr_steps += 1

        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss: {loss_step}")

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Training Loss Epoch {epoch}: {epoch_loss}")

    return

In [ ]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train(epoch)

1it [00:00,  2.96it/s]

Training Loss: 0.020364899188280106


101it [00:35,  2.83it/s]

Training Loss: 0.02004164196942739


201it [01:10,  2.90it/s]

...

Training Loss: 0.016142944498939712


1401it [08:06,  2.88it/s]

Training Loss: 0.0161797491001626


1412it [08:10,  2.88it/s]

Training Loss Epoch 4: 0.016181965652561716


In [ ]:
def test(controlled_model, controlled_test_loader):
    controlled_model.eval()
    tr_loss=0; nb_tr_steps=0;
    total_outputs1 = []
    total_outputs2 = []
    total_targets1 = []
    total_targets2 = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(controlled_test_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs1, outputs2 = controlled_model(ids, mask, token_type_ids)
            outputs1 = outputs1.squeeze()
            outputs2 = outputs2.squeeze()
            total_outputs1.append(outputs1)
            total_outputs2.append(outputs2)
            total_targets1.append(targets[:,0])
            total_targets2.append(targets[:,1])

            loss1 = criterion(outputs1, targets[:,0])
            loss2 = criterion(outputs2, targets[:,1])
            loss_total = loss1 + loss2
            tr_loss += loss_total.item()

            nb_tr_steps += 1

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Validation Loss: {loss_step}")

    total_outputs1=torch.cat(total_outputs1, dim=0)
    total_outputs2=torch.cat(total_outputs2, dim=0)
    total_targets1=torch.cat(total_targets1, dim=0)
    total_targets2=torch.cat(total_targets2, dim=0)
    score1=r2_score(total_outputs1, total_targets1)
    score2=r2_score(total_outputs2, total_targets2)
    print(f"Valence r2_score: {score1}")
    print(f"Arousal r2_score: {score2}")

    epoch_loss = tr_loss/nb_tr_steps
    print(f"Validation Loss Epoch: {epoch_loss}")

    return epoch_loss

In [ ]:
loss = test(controlled_model, controlled_test_loader)
print("Loss on neutral-added test data = %.6f" % loss)

2it [00:00,  7.18it/s]

Validation Loss: 0.04833574593067169


102it [00:11,  8.68it/s]

Validation Loss: 0.09915489909045472


202it [00:23,  8.84it/s]

Validation Loss: 0.09740298240913532


302it [00:34,  8.98it/s]

Validation Loss: 0.09834653276762928


353it [00:40,  8.81it/s]


Valence r2_score: 0.3289760947227478
Arousal r2_score: 0.20022296905517578
Validation Loss Epoch: 0.0967817152873415
Loss on neutral-added test data = 0.096782


In [ ]:
sample_list_v = []
sample_list_a = []
sample_test(controlled_model, sample_loader)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
5it [00:03,  1.99it/s]



1 v score: 0.1112285628914833
1 a score: 0.5369959473609924


2 v score: 0.5219127535820007
2 a score: 0.5694758892059326


3 v score: 0.8105926513671875
3 a score: 0.5642443895339966


4 v score: 0.21347759664058685
4 a score: 0.817337155342102


5 v score: 1.0116418600082397
5 a score: 0.8573595881462097


6 v score: 0.043056149035692215
6 a score: 0.2663394808769226


7 v score: 0.16094917058944702
7 a score: 0.8641214966773987


8 v score: 0.13689427077770233
8 a score: 0.8358659744262695


9 v score: 0.09005885571241379
9 a score: 0.29161664843559265


17it [00:03,  9.30it/s]



10 v score: 0.11514956504106522
10 a score: 0.8404662013053894


11 v score: 0.05161454901099205
11 a score: 0.6585338115692139


12 v score: 0.1422714740037918
12 a score: 0.7015849947929382


13 v score: 0.21509139239788055
13 a score: 0.628960371017456


14 v score: 0.1716366857290268
14 a score: 0.7047050595283508


15 v score: 0.15763358771800995
15 a score: 0.8195429444313049


16 v score: 0.6804817914962769
16 a score: 0.5774851441383362


17 v score: 0.9840561747550964
17 a score: 0.8541523218154907


18 v score: 0.8379753232002258
18 a score: 0.6958123445510864


19 v score: 0.9583017230033875
19 a score: 0.7897192239761353


20 v score: 0.780738115310669
20 a score: 0.4938727617263794


21 v score: 0.8586741089820862
21 a score: 0.5209620594978333


30it [00:03,  7.99it/s]



22 v score: 0.20334652066230774
22 a score: 0.8080248236656189


23 v score: 0.09403298050165176
23 a score: 0.8511219024658203


24 v score: 0.0455167293548584
24 a score: 0.2631983757019043


25 v score: 0.8217437267303467
25 a score: 0.5730892419815063


26 v score: 0.8750919699668884
26 a score: 0.5501427054405212


27 v score: 0.3952573239803314
27 a score: 0.5893611907958984


28 v score: 0.8750919699668884
28 a score: 0.5501427054405212


29 v score: 0.1792362928390503
29 a score: 0.6168680787086487


30 v score: 0.21466004848480225
30 a score: 0.5013167262077332


In [ ]:
print(sample_list_v)
print(sample_list_a)

[-0.7775428742170334, 0.043825507164001465, 0.621185302734375, -0.5730448067188263, 1.0232837200164795, -0.9138877019286156, -0.678101658821106, -0.7262114584445953, -0.8198822885751724, -0.7697008699178696, -0.8967709019780159, -0.7154570519924164, -0.5698172152042389, -0.6567266285419464, -0.6847328245639801, 0.3609635829925537, 0.9681123495101929, 0.6759506464004517, 0.9166034460067749, 0.5614762306213379, 0.7173482179641724, -0.5933069586753845, -0.8119340389966965, -0.9089665412902832, 0.6434874534606934, 0.7501839399337769, -0.20948535203933716, 0.7501839399337769, -0.6415274143218994, -0.5706799030303955]
[0.07399189472198486, 0.13895177841186523, 0.12848877906799316, 0.6346743106842041, 0.7147191762924194, -0.4673210382461548, 0.7282429933547974, 0.6717319488525391, -0.4167667031288147, 0.6809324026107788, 0.31706762313842773, 0.40316998958587646, 0.2579207420349121, 0.40941011905670166, 0.6390858888626099, 0.15497028827667236, 0.7083046436309814, 0.39162468910217285, 0.5794384

In [ ]:
print(np.mean(sample_list_a))

0.2794946372509003


In [ ]:
from datetime import date, datetime

In [ ]:
day = date.today().day
current_time = datetime.now().strftime("%H_%M_%S")

output_model_file = 'emotion_recognizer/models/controlled_model_epoch_10_' + '%d' % day + '_' + current_time + '.pt'
#output_vocab_file = 'emotion_recognizer/'

model_to_save = controlled_model
torch.save(model_to_save, path + output_model_file)
#tokenizer.save_vocabulary(path + output_vocab_file)

print('CONTROLLED_Model saved')

NEU_Model saved
